# Phân tích Nhóm biến Tâm lý - Áp lực

Notebook này tập trung phân tích các biến liên quan đến tâm lý và áp lực của sinh viên, bao gồm:
- Academic Pressure
- Work Pressure
- Study Satisfaction
- Job Satisfaction
- Financial Stress
- Suicidal Thoughts
- Family History of Mental Illness

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Cấu hình hiển thị
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

In [ ]:
# Đọc dữ liệu
df = pd.read_csv("../data/student_depression_dataset.csv")

# Đổi tên cột dài cho dễ xử lý
df = df.rename(columns={"Have you ever had suicidal thoughts ?": "Suicidal Thoughts"})

# Danh sách các biến cần phân tích
psych_pressure_vars = [
    "Academic Pressure",
    "Work Pressure",
    "Study Satisfaction",
    "Job Satisfaction",
    "Financial Stress",
    "Suicidal Thoughts",
    "Family History of Mental Illness",
]

# Hiển thị thông tin cơ bản
df[psych_pressure_vars].info()
display(df[psych_pressure_vars].head())
display(df[psych_pressure_vars].describe())

In [ ]:
# Phân tích phân bố các biến định lượng
num_vars = [
    "Academic Pressure",
    "Work Pressure",
    "Study Satisfaction",
    "Job Satisfaction",
    "Financial Stress",
]

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for i, var in enumerate(num_vars):
    sns.histplot(data=df, x=var, kde=True, ax=axes[i], bins=10)
    axes[i].set_title(f"Phan bo cua {var}")


plt.tight_layout()
plt.show()

In [ ]:
# Phân tích phân bố các biến định tính
cat_vars = ["Suicidal Thoughts", "Family History of Mental Illness"]

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

for i, var in enumerate(cat_vars):
    sns.countplot(data=df, x=var, ax=axes[i])
    axes[i].set_title(f"Phan bo cua {var}")

    # Thêm nhãn số lượng
    for container in axes[i].containers:
        axes[i].bar_label(container)

plt.tight_layout()
plt.show()

In [ ]:
# Làm sạch dữ liệu: Chuyển đổi các biến định lượng sang kiểu số và xử lý giá trị lạ (như '?')
for var in num_vars:
    # Chuyển đổi sang numeric, các giá trị không phải số (như '?') sẽ thành NaN
    df[var] = pd.to_numeric(df[var], errors="coerce")

# Kiểm tra lại số lượng giá trị null sau khi chuyển đổi
print("So luong gia tri thieu sau khi lam sach:")
print(df[num_vars].isnull().sum())

# Lấp đầy giá trị thiếu bằng trung vị (median) hoặc loại bỏ
# Ở đây ta sẽ lấp bằng median để giữ lại dữ liệu
for var in num_vars:
    df[var] = df[var].fillna(df[var].median())

In [ ]:
# Tương quan giữa các biến định lượng
plt.figure(figsize=(10, 8))
correlation = df[num_vars].corr()
sns.heatmap(correlation, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Tuong quan giua cac bien Ap luc va Hai long")
plt.show()

In [ ]:
# In ra các cặp biến có tương quan cao nhất (tuyệt đối) để dễ quan sát
corr_unstack = correlation.unstack()
corr_sorted = corr_unstack.sort_values(kind="quicksort", ascending=False)
print("Top correlations:")
print(corr_sorted[corr_sorted != 1.0].head(10))

## Insights

Dựa trên các biểu đồ và phân tích trên, chúng ta có thể rút ra một số insight ban đầu:

1.  **Phân bố Áp lực**: Biểu đồ histogram cho thấy sự phân bố của Áp lực học tập và Áp lực công việc. Nếu phân phối lệch phải, điều này cho thấy đa số sinh viên chịu áp lực thấp, nhưng có một nhóm nhỏ chịu áp lực rất cao.
2.  **Mối liên hệ giữa Hài lòng và Áp lực**: Biểu đồ nhiệt (heatmap) giúp nhận diện mối tương quan. Thường thì Áp lực (Pressure) sẽ có tương quan âm với Sự hài lòng (Satisfaction).
3.  **Yếu tố nguy cơ**: Tỷ lệ sinh viên có suy nghĩ tự tử (Suicidal Thoughts) và Lịch sử bệnh lý gia đình là các chỉ số quan trọng cần lưu ý.

## Research Questions

Dưới đây là các câu hỏi nghiên cứu, bao gồm cả khía cạnh lý thuyết và ứng dụng thực tế:

1.  **Tác động tích lũy của các loại Áp lực lên Suy nghĩ tự tử**: *Liệu sự kết hợp của Áp lực học tập (Academic Pressure), Áp lực công việc (Work Pressure) và Căng thẳng tài chính (Financial Stress) có làm tăng nguy cơ xuất hiện Suy nghĩ tự tử (Suicidal Thoughts) theo cấp số nhân hay không, và loại áp lực nào đóng vai trò chủ chốt?*
2.  **Vai trò của Sự hài lòng trong việc giảm thiểu rủi ro từ Tiền sử gia đình**: *Đối với những sinh viên có Tiền sử bệnh tâm lý gia đình (Family History of Mental Illness), liệu mức độ Hài lòng trong học tập (Study Satisfaction) hoặc công việc (Job Satisfaction) cao có giúp giảm tỷ lệ có Suy nghĩ tự tử (Suicidal Thoughts) xuống mức tương đương với nhóm không có tiền sử bệnh hay không?*
3.  **Xây dựng "Ngưỡng Báo Động" (Danger Threshold) cho Can thiệp sớm (Thực tế)**: *Tại mức điểm cụ thể nào của Áp lực học tập (Academic Pressure), xác suất sinh viên có suy nghĩ tự tử sẽ vượt qua ngưỡng rủi ro cao (ví dụ: > 50%)? Việc xác định con số này giúp nhà trường thiết lập các bộ lọc tự động để phát hiện và hỗ trợ sinh viên kịp thời.*

# Thực hiện Experiments để trả lời Research Questions

Để trả lời 2 câu hỏi nghiên cứu đã đề ra, chúng ta sẽ thực hiện các phân tích thống kê và trực quan hóa sâu hơn.

## Chuẩn bị dữ liệu
Trước hết, cần chuyển đổi các biến mục tiêu và biến phân loại sang dạng số (0 và 1) để có thể đưa vào mô hình hồi quy và tính toán tỷ lệ.
- `Suicidal Thoughts`: Yes -> 1, No -> 0
- `Family History of Mental Illness`: Yes -> 1, No -> 0

## Phương pháp Phân tích (Methodology)

Để trả lời các câu hỏi nghiên cứu trên, chúng ta sẽ áp dụng các phương pháp phân tích thống kê và trực quan hóa dữ liệu cụ thể như sau:

### RQ1: Tác động tích lũy của các loại Áp lực
- **Phương pháp**: Hồi quy Logistic (Logistic Regression).
- **Lý do**: Biến mục tiêu `Suicidal Thoughts` là biến nhị phân (Yes/No -> 1/0). Hồi quy Logistic là công cụ tiêu chuẩn để mô hình hóa xác suất xảy ra của một sự kiện nhị phân dựa trên các biến độc lập liên tục hoặc rời rạc.
- **Biến độc lập (Independent Variables)**: `Academic Pressure`, `Work Pressure`, `Financial Stress`.
- **Biến phụ thuộc (Dependent Variable)**: `Suicidal Thoughts` (đã mã hóa thành 0 và 1).
- **Kỹ thuật bổ sung**: Tạo biến tổng hợp `Total Pressure` (tổng của 3 loại áp lực) để trực quan hóa xu hướng tích lũy bằng biểu đồ hồi quy (Regplot), giúp dễ dàng quan sát mối quan hệ tuyến tính giữa tổng áp lực và xác suất có suy nghĩ tự tử.

### RQ2: Vai trò của Sự hài lòng và Tiền sử gia đình
- **Phương pháp**: Phân tích So sánh và Trực quan hóa Tương tác (Comparative Analysis & Interaction Visualization).
- **Lý do**: Cần xem xét sự thay đổi của tỷ lệ mục tiêu (`Suicidal Thoughts`) theo mức độ của một biến liên tục (`Satisfaction`) nhưng được phân tách theo các nhóm khác nhau (`Family History`).
- **Công cụ**: Biểu đồ Pointplot (hoặc Lineplot) kèm theo khoảng tin cậy (Confidence Interval - CI 95%).
- **Cách đọc kết quả**: Chúng ta sẽ quan sát độ dốc của các đường biểu diễn cho hai nhóm (Có và Không có tiền sử bệnh). Nếu đường biểu diễn của nhóm "Có tiền sử" có xu hướng đi xuống và tiệm cận với nhóm "Không có tiền sử" khi mức độ hài lòng tăng lên, điều đó cho thấy sự hài lòng có tác dụng giảm thiểu rủi ro. Khoảng tin cậy giúp xác định xem sự khác biệt giữa các điểm dữ liệu có ý nghĩa thống kê hay không.

### RQ3: Xác định "Ngưỡng Báo Động" (Threshold Analysis)
- **Phương pháp**: Phân tích Dự báo (Predictive Analysis) dựa trên mô hình Logistic.
- **Cách thực hiện**: Sử dụng các hệ số hồi quy đã tính toán ở RQ1 để dự báo xác suất có suy nghĩ tự tử tại từng mức điểm của `Academic Pressure` (từ 0 đến 5), trong khi giữ các biến khác ở mức trung bình.
- **Mục tiêu**: Tìm ra điểm cắt (cut-off point) mà tại đó xác suất vượt quá 50%. Đây sẽ là "ngưỡng báo động đỏ" cho các nhà quản lý giáo dục.

In [ ]:
# Chuyển đổi biến mục tiêu sang dạng số
df["Suicidal Thoughts_Num"] = df["Suicidal Thoughts"].map({"Yes": 1, "No": 0})
df["Family History_Num"] = df["Family History of Mental Illness"].map(
    {"Yes": 1, "No": 0}
)

# Kiểm tra lại dữ liệu sau khi chuyển đổi
print(
    df[
        [
            "Suicidal Thoughts",
            "Suicidal Thoughts_Num",
            "Family History of Mental Illness",
            "Family History_Num",
        ]
    ].head()
)

## RQ1: Tác động tích lũy của các loại Áp lực lên Suy nghĩ tự tử

Chúng ta sẽ sử dụng mô hình Hồi quy Logistic để đánh giá tác động của từng loại áp lực (Học tập, Công việc, Tài chính) lên xác suất có suy nghĩ tự tử. Đồng thời, ta sẽ tạo một biến tổng hợp `Total Pressure` để xem xét tác động tích lũy.

In [ ]:
import statsmodels.api as sm

# 1. Mô hình Hồi quy Logistic đa biến
X = df[["Academic Pressure", "Work Pressure", "Financial Stress"]]
X = sm.add_constant(X)  # Thêm hệ số chặn
y = df["Suicidal Thoughts_Num"]

# Xử lý NaN nếu còn sót lại (dù đã fillna ở trên, kiểm tra lại cho chắc chắn)
if X.isnull().values.any():
    X = X.fillna(X.median())

model = sm.Logit(y, X)
result = model.fit()
print(result.summary())

# 2. Trực quan hóa tác động tích lũy
# Tạo biến Total Pressure
df["Total Pressure"] = (
    df["Academic Pressure"] + df["Work Pressure"] + df["Financial Stress"]
)

plt.figure(figsize=(10, 6))
sns.regplot(
    x="Total Pressure",
    y="Suicidal Thoughts_Num",
    data=df,
    logistic=True,
    ci=95,
    line_kws={"color": "red"},
)
plt.title("Xac suat co Suy nghi tu tu theo Tong muc do Ap luc")
plt.ylabel("Probability of Suicidal Thoughts")
plt.xlabel("Total Pressure (Academic + Work + Financial)")
plt.show()

## RQ2: Vai trò của Sự hài lòng trong việc giảm thiểu rủi ro từ Tiền sử gia đình

Chúng ta sẽ so sánh tỷ lệ có suy nghĩ tự tử giữa hai nhóm sinh viên (có và không có tiền sử bệnh gia đình) tại các mức độ hài lòng khác nhau. Nếu đường biểu diễn của nhóm "Có tiền sử" tiệm cận với nhóm "Không có tiền sử" khi mức độ hài lòng tăng lên, điều đó cho thấy vai trò giảm thiểu rủi ro của sự hài lòng.

In [ ]:
# Trực quan hóa mối quan hệ tương tác
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Study Satisfaction
sns.pointplot(
    x="Study Satisfaction",
    y="Suicidal Thoughts_Num",
    hue="Family History of Mental Illness",
    data=df,
    ax=axes[0],
    capsize=0.1,
    errorbar=("ci", 95),
)
axes[0].set_title(
    "Tac dong cua Study Satisfaction va Family History len Suy nghi tu tu"
)
axes[0].set_ylabel("Ty le co Suy nghi tu tu")

# Job Satisfaction
sns.pointplot(
    x="Job Satisfaction",
    y="Suicidal Thoughts_Num",
    hue="Family History of Mental Illness",
    data=df,
    ax=axes[1],
    capsize=0.1,
    errorbar=("ci", 95),
)
axes[1].set_title("Tac dong cua Job Satisfaction va Family History len Suy nghi tu tu")
axes[1].set_ylabel("Ty le co Suy nghi tu tu")

plt.tight_layout()
plt.show()

## RQ3: Xác định "Ngưỡng Báo Động" cho Áp lực học tập

Chúng ta sẽ tính toán xác suất dự báo có suy nghĩ tự tử khi `Academic Pressure` tăng dần, giả định các yếu tố khác (Work Pressure, Financial Stress) ở mức trung bình. Điều này giúp cô lập tác động của áp lực học tập để tìm ra ngưỡng nguy hiểm.

In [ ]:
import numpy as np

# Tạo dữ liệu giả định để dự báo
# Academic Pressure chạy từ min đến max (ví dụ 0 đến 5)
academic_range = np.linspace(
    df["Academic Pressure"].min(), df["Academic Pressure"].max(), 100
)

# Các biến khác giữ ở mức trung bình (mean) hoặc trung vị (median)
mean_work = df["Work Pressure"].mean()
mean_financial = df["Financial Stress"].mean()

# Tạo DataFrame dự báo
X_pred = pd.DataFrame(
    {
        "const": 1.0,  # Hệ số chặn
        "Academic Pressure": academic_range,
        "Work Pressure": mean_work,
        "Financial Stress": mean_financial,
    }
)

# Dự báo xác suất
y_pred_prob = result.predict(X_pred)

# Tìm ngưỡng (Threshold) nơi xác suất > 0.5
threshold_idx = np.where(y_pred_prob > 0.5)[0]
if len(threshold_idx) > 0:
    threshold_value = academic_range[threshold_idx[0]]
    print(
        f"Ngưỡng báo động (Probability > 50%): Academic Pressure = {threshold_value:.2f}"
    )
else:
    threshold_value = None
    print("Trong phạm vi dữ liệu hiện tại, xác suất trung bình không vượt quá 50%.")

# Trực quan hóa
plt.figure(figsize=(10, 6))
plt.plot(
    academic_range, y_pred_prob, label="Predicted Probability", color="red", linewidth=2
)
plt.axhline(y=0.5, color="grey", linestyle="--", label="Risk Threshold (50%)")

if threshold_value:
    plt.axvline(
        x=threshold_value,
        color="orange",
        linestyle="--",
        label=f"Threshold: {threshold_value:.2f}",
    )
    plt.scatter([threshold_value], [0.5], color="black", zorder=5)

plt.title("Du bao Xac suat Suy nghi tu tu theo Ap luc Hoc tap")
plt.xlabel("Academic Pressure")
plt.ylabel("Probability")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Kết luận

Dựa trên kết quả thực nghiệm, chúng ta có thể đưa ra câu trả lời chi tiết cho các câu hỏi nghiên cứu:

### Trả lời RQ1: Tác động của Áp lực
**Kết luận**: Có sự tác động tích lũy rõ rệt. Khi tổng mức độ áp lực tăng, xác suất xuất hiện suy nghĩ tự tử tăng lên đáng kể.

**Phương pháp & Phân tích**:
-   **Phương pháp**: Sử dụng **Hồi quy Logistic** để mô hình hóa mối quan hệ giữa các biến áp lực (độc lập) và biến suy nghĩ tự tử (phụ thuộc, nhị phân).
-   **Bằng chứng thống kê**:
    -   **Academic Pressure**: Hệ số hồi quy (coef) là **0.3774** với P-value **0.000** (< 0.05). Điều này có nghĩa là áp lực học tập có tác động dương và rất có ý nghĩa thống kê lên khả năng có suy nghĩ tự tử.
    -   **Financial Stress**: Hệ số là **0.2695** với P-value **0.000**. Đây cũng là một yếu tố dự báo mạnh mẽ.
    -   **Work Pressure**: Hệ số là **0.1888** nhưng P-value là **0.499** (> 0.05). Do đó, trong tập dữ liệu này, áp lực công việc không có ý nghĩa thống kê (có thể do phần lớn sinh viên không đi làm).
-   **Bằng chứng trực quan**: Biểu đồ hồi quy (`regplot`) cho thấy đường cong logistic đi lên rõ rệt, khẳng định mối quan hệ thuận chiều giữa tổng áp lực và rủi ro.

### Trả lời RQ2: Vai trò của Sự hài lòng
**Kết luận**: **Sự hài lòng trong học tập (Study Satisfaction)** đóng vai trò là một **yếu tố bảo vệ (protective factor)** quan trọng, đặc biệt đối với nhóm sinh viên có nguy cơ cao.

**Phương pháp & Phân tích**:
-   **Phương pháp**: **Phân tích So sánh và Trực quan hóa Tương tác** (Pointplot với khoảng tin cậy 95%) để so sánh tỷ lệ rủi ro giữa hai nhóm (có và không có tiền sử bệnh) theo các mức độ hài lòng khác nhau.
-   **Study Satisfaction**:
    -   **Xu hướng**: Đường màu xanh (Nhóm có tiền sử bệnh) đi xuống rõ rệt khi mức độ hài lòng tăng.
    -   **Sự hội tụ**: Tại mức hài lòng thấp (0-1), khoảng cách rủi ro giữa hai nhóm rất lớn. Tuy nhiên, khi mức độ hài lòng đạt đỉnh (5), hai đường này gặp nhau. Điều này chứng minh sự hài lòng cao có thể "bù đắp" cho rủi ro từ tiền sử gia đình.
-   **Job Satisfaction**: Kết quả không rõ ràng do dữ liệu thưa thớt (khoảng tin cậy quá rộng), không đủ cơ sở để kết luận.

### Trả lời RQ3: Ngưỡng Báo Động (Thực tế)
**Kết luận**: Ngưỡng báo động cho **Áp lực học tập** là **1.57** (trên thang điểm 5).

**Phương pháp & Phân tích**:
-   **Phương pháp**: **Phân tích Dự báo (Predictive Analysis)** kết hợp kỹ thuật **"Ceteris Paribus"** (Giữ nguyên các yếu tố khác). Chúng ta cố định các biến khác (*Tài chính, Công việc*) ở mức trung bình và chỉ cho biến *Áp lực học tập* chạy từ 0 đến 5 để cô lập tác động của nó.
-   **Điểm cắt (Cut-off Point)**: Tìm giao điểm nơi đường cong xác suất dự báo cắt ngang mức rủi ro 50%.
-   **Ý nghĩa thực tiễn**: Khi sinh viên báo cáo mức độ áp lực học tập >= **1.6**, xác suất họ có suy nghĩ tự tử đã vượt quá **50%**.
-   **Khuyến nghị**: Nhà trường nên thiết lập hệ thống cảnh báo tự động. Bất kỳ sinh viên nào có điểm `Academic Pressure >= 2` nên được đưa vào danh sách ưu tiên hỗ trợ tâm lý ngay lập tức.